# f2pyを使う

f2pyはfortranのプログラムをpythonから呼びだすためのツールです。両方使えると、互いの弱点を補うことができます。

Anacondaにはnumpyが標準で含まれており、numpyはf2pyをインストールしてくれる、ということなのですが、Windowsの場合、fortranが標準で使えるようになっているとは思えません。一応、その方法を書いてあるページをリンクしておきます。

* http://qiita.com/tenomoto/items/57f46652ecef1ee03c10

実際には、fortranを自分の端末ではなく計算サーバ上で使う場合のほうが多いでしょうから、自分の端末で以下の作業ができるようにする必要はないと思います。

F2py is a utility to use the fortran code from Python.  They are complementary in speed and easiness of programming.

Anaconda contains numpy and numpy accompanies f2py.  However, I cannot believe that Anaconda on Windows enables the use of fortran automatically.

In actual use, you would use the fortran on the workstations, so you need not make effort to let them work on your own PC.

Pythonの弱点はスピードです。

The weak point of Python language is its speed.

例えば、1から100000000までの平方根の総和の計算をさせてみると、10秒以上かかります。

For example, it takes more than 10 seconds to calculate the sum of square roots of the numbers from 1 to 100000000.

In [2]:
import time

def sqrt_sum(n):
    sum = 0.0
    for i in range(n+1):
        sum += i**0.5
    return sum

now = time.time()
print(sqrt_sum(100000000))
print(time.time()-now, "elapsed time")


666666671666.567
12.242816925048828 elapsed time


同じ計算を、以下のようにFortranで書けば、1秒ほどで終わります。

The same calculation requires only one second by fortran.

(**NOTE**: the following code is written in fortran. Put it in separate file named `test1.f90`)

In [ ]:
subroutine sqrt_sum(n, sum)
  implicit none

  integer, intent(IN) :: n
  real(kind=8), intent(OUT) :: sum

  integer :: i

  sum = 0.0
  do i=1,n
    sum = sum + i**0.5
  end do
end subroutine sqrt_sum

program main
  real(kind=8) :: sum
  call sqrt_sum(100000000, sum)
  print *, sum
end program main

そこで、単純だけど時間のかかる計算の部分だけをfortranにまかせ、ほかのこまごました処理は全部Pythonでできれば幸せになれます。

Therefore, it would be happy if we can write the simple but heavy calculation in fortran and combine it with Python to do other things.

上のfortranプログラムの名前をtest1.f90とします。これをpythonから呼びだせるようにするには、f2pyコマンドを使います。

The program above, say `test1.f90` is converted to a module for Python by the following command:

```
% f2py -c test1.f90 -m test1_mod
```

test1_modはPythonから呼びだす時のモジュール名です。これをPythonから呼びだすのはとても簡単です。

Here `test1_mod` is the module name when it is imported to Python.  It is very easy to use.

In [3]:
import time
import test1_mod

now = time.time()
print(test1_mod.sqrt_sum(100000000))
print(time.time()-now, "elapsed time")

666666671663.059
1.4525971412658691 elapsed time


なんか簡単すぎて拍子抜けします。fortranではfunction(返り値がある)ではなくsubroutineとして定義したのですが、f2pyがsqrt_sumの引数を(intent指示をもとに)解釈し、pythonらしく使えるようにしてくれたようです。つまり、f2pyはただ単にfortranをPythonから呼べるようにするだけではなく、それぞれのプログラミングスタイルの橋渡しもしてくれるようです。

It just works without any trouble.  In fortran, we defined a subroutine instead of a function, but f2py assumed that is a function with the return value `sum`.  In other words, f2py also converts the programming style.

f2pyがどのようにinterfaceを作ってくれたのかは、`__doc__`を参照して下さい。

You can check the interface of the module by pringing the `__doc__` variable of the module.

In [30]:
print(test1_mod.__doc__)

This module 'test1_mod' is auto-generated with f2py (version:2).
Functions:
  sum = sqrt_sum(n)
.


確かに、sqrt_sumは2引数のsubroutineとして定義されているが、実質的にはfunctionであると理解していることがわかります。

Actually, `sqrt_sum` is regarded as a function with single argument and single return value.

試しに、2つの値を返すsubroutineを作ってみます。fortranでは複数の値を返すfunctionは作れないので、subroutineにするしかありませんが、pythonは複数の値を返すことができます。なので、2つの言語では、だいぶ書き方が違ってきます。

As another test, let's make a subroutine that returns two values at a time.  In fortran, it is impossible to make a function that returns multiple values.  In Python, on the other hand, it is easy to return two values from a function.

(**NOTE**: the following code is written in fortran. Put it in separate file named `test2.f90`)

In [ ]:
subroutine division(a,b,c,d)
  implicit none

  integer, intent(IN) :: a,b
  integer, intent(OUT) :: c,d

  c = a / b
  d = mod(a,b)
end subroutine division

program main
  integer :: c,d
  call division(10,7,c,d)
  print *, c,d
end program main

上のプログラムをtest2.f90という名前で保存し、f2pyでモジュールを作成します。

Save the program above in a file named `test2.f90` and make the module for Python by f2py command.

```
% f2py -c test2.f90 -m test2_mod
```

サブルーチンdivisionがどのように解釈されたか見てみましょう。

Let's see how the division subroutine is converted to a Python function.

In [31]:
import test2_mod
print(test2_mod.__doc__)

This module 'test2_mod' is auto-generated with f2py (version:2).
Functions:
  c,d = division(a,b)
.


4つの引数のうち2つは返り値と認識され、Pythonらしい形に変換されたことがわかります。実際に動くかどうか確かめましょう。

Two arguments out of four of the fortan subroutines are recognized as the return value, and in fact the function returns two values.

In [6]:
test2_mod.division(10,7)

(1, 3)

商は1、余りが3と正しく計算できています。

The quotient is 1 and remainder is 3.

もっと大量のデータを扱う場合には、numpy arrayを渡して計算させたくなります。もちろん、numpy自体も高速なので、たいていのことはnumpyだけで片付きますが、numpyでできない処理がでてきた場合、Pythonとのあまりの速度差に不満を感じることになります。

例えば、動径分布関数の計算をnumpyの機能だけでなんとかしようとすると、かなりの無駄が生じますが、Pythonだけで記述するとかなり遅くなります。こういう場合には、中核部分だけをfortranにまかせるのが最適です。

まず、単純立方格子を生成します。(これも多重ループで時間がかかりますね。numpyで簡潔に書く方法があると思います)

In [10]:
import numpy

def lattice(n):
    pos = numpy.zeros((n**3,3))
    i = 0
    for x in range(n):
        for y in range(n):
            for z in range(n):
                p = numpy.array([x,y,z])
                pos[i,:] = p
                i += 1
    return pos

lattice(2)                

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  1.],
       [ 1.,  1.,  0.],
       [ 1.,  1.,  1.]])

動径分布関数の本質は、2体間距離の分布関数(ヒストグラム)です。なので、距離分布関数を、Pythonで書きます。

distribは、LxLxLの立方体セル(周期境界条件)の中の原子の座標posを読みこみ、距離dmaxまでの分布関数を、単位距離あたりndiv分割した目盛で作成します。

In [20]:
import itertools
import time

def distrib(pos, L, dmax=5, ndiv=10):
    d = numpy.zeros(dmax*ndiv)
    # combination of two elements in pos
    for p1,p2 in itertools.combinations(pos,2):
        # relative vector
        dp = p1-p2
        
        # treatment for the periodic boundary condition
        dp -= numpy.floor( dp / L + 0.5 ) * L
        
        # distance
        r = numpy.dot(dp, dp)**0.5

        # make it an integer
        ir = int(r*ndiv)
        
        # Accumulate if the distance is within the range
        if ir < dmax*ndiv:
            d[ir] += 1
    return d

now = time.time()
N = 20
print(distrib(lattice(N), N))
print(time.time() - now)

[      0.       0.       0.       0.       0.       0.       0.       0.
       0.       0.   24000.       0.       0.       0.   48000.       0.
       0.   32000.       0.       0.   24000.       0.   96000.       0.
   96000.       0.       0.       0.   48000.       0.  120000.   96000.
       0.   96000.   32000.       0.   96000.  192000.       0.       0.
   24000.  192000.  144000.   96000.   96000.  192000.   96000.       0.
   96000.       0.]
325.95744609832764


これを動径分布関数に変換するには、いくつかの規格化を行う必要がありますが、ここでは説明しません。

たった8000点の計算でもうんざりするぐらい時間がかかります。そこで、distribをfortranで書いてみることにしましょう。

(**NOTE**: the following code is written in fortran. Put it in separate file named `test3.f90`)

In [ ]:
subroutine lattice(L, pos)
  implicit none

  integer, intent(IN)       :: L
  real(kind=8), intent(OUT) :: pos(L*L*L,3)

  integer :: n,i,j,k

  n = 0
  do i=1,L
     do j=1,L
        do k=1,L
           n = n + 1
           pos(n,1) = i
           pos(n,2) = j
           pos(n,3) = k
        end do
     end do
  end do
end subroutine lattice


subroutine distrib(N, pos, L, dmax, ndiv, d)
  implicit none

  integer, intent(IN)      :: N
  real(kind=8), intent(IN) :: pos(N,3)
  real(kind=8), intent(IN) :: L
  integer, intent(IN)      :: ndiv, dmax
  real(kind=8), intent(OUT) :: d(dmax*ndiv)
  
  integer      :: i, j, ir
  real(kind=8) :: p1(3), p2(3), dp(3), r
  
  do i=1,N
     p1(:) = pos(i,:)
     do j=i+1, N
        p2(:) = pos(j,:)
        dp(:) = p1(:) - p2(:)
        dp(:) = dp(:) - dnint(dp(:) / L)*L
        r = (dp(1)**2 + dp(2)**2 + dp(3)**2) ** 0.5
        ir = int(r*ndiv)
        if ( ir < dmax*ndiv ) then
           d(ir+1) = d(ir+1) + 1
        end if
     end do
  end do
end subroutine distrib


program main
  implicit none

  integer, parameter :: L=20
  real(kind=8) :: pos(L**3, 3)
  integer :: i
  real(kind=8) :: LL, d(50)

  LL = L
  call lattice(L, pos)
  call distrib(L**3, pos, LL, 5, 10, d)
  do i=1,50
     print *, i-1,d(i)
  end do
end program main


このまま実行すると、おおよそ200倍ぐらい速く実行されることがわかります。

```
% gfortran -O test3.f90 -o test3
% time ./test3

real	0m1.711s
user	0m1.701s
sys	0m0.006s
```

そこで、このプログラムをf2pyで加工して、pythonモジュールにしてしまいましょう。

```
% f2py -c test3.f90 -m test3_mod
```

できたモジュールを読みこみ、`__doc__`を表示させて、どんな風に解釈されたかを見てみます。

In [21]:
import test3_mod
print(test3_mod.__doc__)

This module 'test3_mod' is auto-generated with f2py (version:2).
Functions:
  pos = lattice(l)
  d = distrib(pos,l,dmax,ndiv,n=shape(pos,0))
.


なかなかいいかんじです。では、このinterfaceを信じて、まずはlatticeをpythonで呼びだしてみることにしましょう。

In [27]:
import test3_mod

L = 20
pos = test3_mod.lattice(L)
pos

array([[  1.,   1.,   1.],
       [  1.,   1.,   2.],
       [  1.,   1.,   3.],
       ..., 
       [ 20.,  20.,  18.],
       [ 20.,  20.,  19.],
       [ 20.,  20.,  20.]])

なんと、結果はnumpy arrayで返ってきています。これはいいですね。では、さらにdistribを呼んで分布関数を計算させてみます。

distribの5番目の引数nは、fortranの中ではpos配列の大きさを指示していたのですが、f2pyしたあとは指示しなくても勝手によきにはからってくれるようです。

In [28]:
d = test3_mod.distrib(pos,L,5,10)
d

array([      0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,   24000.,       0.,
             0.,       0.,   48000.,       0.,       0.,   32000.,
             0.,       0.,   24000.,       0.,   96000.,       0.,
         96000.,       0.,       0.,       0.,   48000.,       0.,
        120000.,   96000.,       0.,   96000.,   32000.,       0.,
         96000.,  192000.,       0.,       0.,   24000.,  192000.,
        144000.,   96000.,   96000.,  192000.,   96000.,       0.,
         96000.,       0.])

はやっ! 試しに時間をはかってみます。

In [29]:
now = time.time()
d = test3_mod.distrib(pos,L,5,10)
print(time.time() - now)

1.5317890644073486


fortranのみで書いた場合とほぼ同じ時間で処理がおわりました。

Pythonは文字列を処理したり、複雑な構造のデータを扱ったり、こみいったアルゴリズムを記述するのに向いています。また、ライブラリも非常に充実していて、開発にかかる時間も非常に短くてすみます。

一方で、単純だけど膨大な数値データの処理にはあまり向いていません。numpyはその一部を高速に処理してくれますが、すべてのアルゴリズムがnumpyに向いているわけではありません。

f2pyを使うと、Pythonの弱点を補ってくれます。

ですから、ここでデモしたように、はじめPythonでまず書いておいて、その中で一番処理に時間がかかる部分で、fortranで書けそうな部分をfortranで書きなおし、f2pyで合体させて使う、というやりかたはとても効率的だと思います。